# ID2214/FID3214 Assignment 2 Group no. [enter]
### Project members: 
[Patrik Zhong, pzhong@kth.se]
[Edvin Walleborn, edvinw@kth.se]
[Alexander Carlsson, alecarls@kth.se]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.9.13
NumPy version: 1.21.5
Pandas version: 1.4.4


## Reused functions from Assignment 1

In [3]:
# Copy and paste functions from Assignment 1 here that you need for this assignment


globalTestList = []
def create_column_filter(df): #kommer ej fungera, sök annan approach!
    dfnew = df.copy()    
    column_filter = []
    for col in dfnew.columns:
        if col == 'CLASS' or col == 'ID':
            column_filter.append(col)
            continue
        
        if dfnew[col].isna().all():
            dfnew.drop(labels=col, axis=1, inplace = True)
        
        elif(dfnew[col].dropna().nunique() <= 1):
            dfnew.drop(col, axis = 1, inplace = True)
        
        else:
            column_filter.append(col)
        
  
        
    return dfnew, column_filter

def apply_column_filter(df, column_filter):
    dfnew = df.copy()
    dfnew= dfnew.filter(items = column_filter, axis = 1)

    return dfnew


def create_normalization(df, normalizationtype):
    dfnew = df.copy()
    #display(dfnew)
    valueDict = {}


    if normalizationtype == 'zscore':
        for col in dfnew.columns:
            
            not_float = not pd.api.types.is_float_dtype(dfnew[col].dtypes)
            not_int64 = not pd.api.types.is_int64_dtype(dfnew[col].dtypes)

            if col == 'CLASS' or col == 'ID' or not_float and not_int64:
                continue
       
            meanVal = dfnew[col].mean()
            stdVal = dfnew[col].std()
            dfnew[col] = dfnew[col].apply(lambda x: (x-meanVal)/stdVal)
            valueDict[col] = ("zscore", meanVal, stdVal)

    elif normalizationtype == 'minmax':
        for col in dfnew.columns:
                
            not_float = not pd.api.types.is_float_dtype(dfnew[col].dtypes)
            not_int64 = not pd.api.types.is_int64_dtype(dfnew[col].dtypes)

            if col == 'CLASS' or col == 'ID' or not_float and not_int64:
                continue
           
            minVal = dfnew[col].min()
            maxVal = dfnew[col].max()
            # for count, value in enumerate(dfnew[col]):
            #     dfnew[col][count] = (value-minVal)/(maxVal-minVal) 
            dfnew[col] = dfnew[col].apply(lambda value: (value-minVal)/(maxVal-minVal)) 
            valueDict[col] = ("minmax", minVal, maxVal) 
                           

    else: 
        print("Normalization type not recognized")

    # print(valueDict)
    #display(dfnew)
    #print(valueDict)

    return dfnew, valueDict

def apply_normalization(df, normalization):
    dfnew = df.copy()
    for col in dfnew.columns:
        not_float = not pd.api.types.is_float_dtype(dfnew[col].dtypes)
        not_int64 = not pd.api.types.is_int64_dtype(dfnew[col].dtypes)

        if col == 'CLASS' or col == 'ID' or not_float and not_int64:
            continue

        normalizationType = normalization[col][0] #minmax eller ztype       

        if normalizationType == "minmax":          
            minVal = normalization[col][1]
            maxVal = normalization[col][2]
            dfnew[col] = dfnew[col].apply(lambda value: (value-minVal)/(maxVal-minVal))
            dfnew[col] = dfnew[col].apply(lambda x: 0.0 if x<0.0 else (1.0 if x > 1.0 else x))

        if normalizationType == "zscore":
            meanVal = normalization[col][1]
            stdVal = maxVal = normalization[col][2]
            dfnew[col] = df[col].apply(lambda x: (x-meanVal)/stdVal)

    return dfnew

def create_imputation(df):
    dfnew = df.copy()
    mappingDict = {}
    #display(dfnew)
    for col in dfnew.columns:
        is_float = pd.api.types.is_float_dtype(dfnew[col].dtypes)
        is_int64 = pd.api.types.is_int64_dtype(dfnew[col].dtypes)
        is_object = pd.api.types.is_object_dtype(dfnew[col].dtypes)
        is_category = pd.api.types.is_categorical_dtype(dfnew[col].dtypes)

        #print(col)
        #print(dfnew[col].dtypes)

        if col == 'CLASS' or col == 'ID':
            continue
        elif (is_int64):
            mean = dfnew[col].mean()
            if pd.isna(mean):
                mean = 0
            dfnew[col].fillna(value=mean, inplace=True)
            mappingDict[col] = (mean)
        elif(is_float):
            mean = dfnew[col].mean()
            if pd.isna(mean):
                mean = 0.0
            dfnew[col].fillna(value=mean, inplace=True)
            mappingDict[col] = (mean)
        elif (is_object):
            mode = dfnew[col].mode(dropna=True)[0]
            if pd.isna(mode):
                mode = ""
            dfnew[col].fillna(value=mode, inplace=True)
            mappingDict[col] = (mode)
        elif(is_category):
            mode = dfnew[col].mode(dropna=True)[0]
            if pd.isna(mode):
                mode = dfnew[col].categories[0]
            dfnew[col].fillna(value=mode, inplace=True)
            mappingDict[col] = (mode)
        else:
            print("no match on dtype in if-statement in (create_imputation)")

    #display(dfnew)
    return dfnew, mappingDict


def apply_imputation(df, imputation):
        dfnew = df.copy()
        for col in dfnew.columns:
            if col == 'CLASS' or col == 'ID':
                continue
            else:
                dfnew[col].fillna(value=imputation[col], inplace=True)
        return dfnew

def find_bin(x, binsVar):

    for i, binn in enumerate(binsVar):
        if x > binn and x <= binsVar[i+1]:
            #print(i)
            if not i in globalTestList:
                globalTestList.append(i)
            return i

def create_bins(df, nobins, bintype):
    dfnew = df.copy()
    binning = {}
    
    #display(dfnew)

    for col in dfnew.columns:

        not_float = not pd.api.types.is_float_dtype(dfnew[col].dtypes)
        not_int64 = not pd.api.types.is_int64_dtype(dfnew[col].dtypes)

        if col == 'CLASS' or col == 'ID' or not_float and not_int64:
            #Discretize is ignored
            continue

        res = None
        binsVar = None
        #Discretize columns from here on
        if bintype == "equal-width":
            res, binsVar = pd.cut(dfnew[col], bins=nobins, labels=False, retbins=True)
        elif bintype == "equal-size":
            #duplicates are set to drop due to error. discuss and lookup
            res, binsVar = pd.qcut(dfnew[col], q=nobins, duplicates="drop", labels=False, retbins=True)
        else:
            print("Not recognizable bintype to create_bins")
        
        binsVar[0] = -np.inf
        binsVar[-1] = np.inf
        binning[col] = binsVar 
        
        #prints to show hint achieved
        #print(res[0])
        #print("res:\n{}".format(res))
        #print("====\nbinsVar:\n{}\n======done=======".format(binsVar))

        dfnew[col] = dfnew[col].apply(lambda x: find_bin(x, binsVar))
        dfnew[col] = dfnew[col].astype("category")

        #print to show hint achieved
        #print(dfnew[col].dtypes)
    #display(dfnew)

    #print to show binning format correct
    #print(binning)
    
    #prints to show hint achieved
    #globalTestList.sort()
    #print(globalTestList)

    return dfnew, binning

def apply_bins(df, binning):
    #hint1
    dfnew = df.copy()

    for col in dfnew.columns:

        not_float = not pd.api.types.is_float_dtype(dfnew[col].dtypes)
        not_int64 = not pd.api.types.is_int64_dtype(dfnew[col].dtypes)

        if col == 'CLASS' or col == 'ID' or not_float and not_int64:
            #Discretize is ignored
            continue

        #hint2
        res, binsVar = pd.cut(dfnew[col], bins=binning[col], labels=False, retbins=True)
        
        #apply bins to output instructions
        dfnew[col] = dfnew[col].apply(lambda x: find_bin(x, binsVar))

        #hint3
        dfnew[col] = dfnew[col].astype("category")

        #hint4 already achieved. see print
        #print("has0: {}\nhas1: {}\nhas2: {}\nhas3: {}\nhas4: {}\nhas5: {}\nhas6: {}\nhas7: {}\nhas8: {}\nhas9: {}\nmin: {}\nmax: {}".format(0 in res, 1 in res, 2 in res, 3 in res, 4 in res, 5 in res, 6 in res, 7 in res, 8 in res, 9 in res, res.min(), res.max()))

    return dfnew

def create_one_hot(df):
    dfnew = df.copy()
    catDict = {}
    # display(dfnew)
    
    for col in dfnew.columns:
        not_object = not pd.api.types.is_object_dtype(dfnew[col].dtypes)
        not_category = not pd.api.types.is_categorical_dtype(dfnew[col].dtypes)
        if col == 'CLASS' or col == 'ID' or not_object and not_category:
            continue
        #turns out that almost all columns are objects, not categories. Cast objects to categories with astype.
        # categories = pd.Categorical(dfnew[col])
        dfnew[col] = dfnew[col].astype("category")
        #varje col får tre nya kolumner för varje kategori (b, o, x)
        dummyVars = pd.get_dummies(dfnew[col])
        #appenda col + [b, o, x] som en egen kolumn
        catList = []
        for catcol in dummyVars:
            catList.append(catcol)
            newName = col + "-" + catcol
            dfnew[newName] = dummyVars[catcol]
            dfnew[newName] = dfnew[newName].astype('float')
        #drop the column after worked through the categories
        dfnew = dfnew.drop(col, axis=1)
    #convert everything to float
        catDict[col] = catList
           
    # display(dfnew)
    # print(catDict)        
    
    return dfnew, catDict

def apply_one_hot(df, one_hot):
    dfnew = df.copy()

    # for col in dfnew.columns

    #     dfnew[col] = dfnew[col].astype("category")

    for col in one_hot: #för varje key i vår dict
        not_object = not pd.api.types.is_object_dtype(dfnew[col].dtypes)
        not_category = not pd.api.types.is_categorical_dtype(dfnew[col].dtypes)
        if col == 'CLASS' or col == 'ID' or not_object and not_category:
            continue
        dummyVars = pd.get_dummies(dfnew[col]) #skapa dummyvariablerna
        for catcol in one_hot[col]: #för varje kategori, appenda kategorin med dummyvariabeln
            newName = col + "-" + catcol
            dfnew[newName] = dummyVars[catcol].astype('float')
        dfnew = dfnew.drop(col, axis=1)


    return dfnew

def split(df, testfraction):

    dfIndexList = np.random.permutation(df.index)

    #testfraction = 0.3
    #dfIndexList = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

    #print(testfraction)
    fracslice = (1-testfraction)*len(dfIndexList)
    #print(fracslice)

    # assumption for special case of ratio resulting in .5 index. round up index instead of default round down
    if fracslice % 1 == 0.5:
        fracslice = round(fracslice + 0.01)
    else:
        fracslice = round(fracslice)
    
        
    #print(fracslice)


    trainingInterval = dfIndexList[:fracslice]
    testInterval = dfIndexList[fracslice:]

    #print("train: {}\ntest: {}\n".format(trainingInterval, testInterval))

    trainingdf = df.filter(items = trainingInterval, axis = 0)
    testdf = df.filter(items = testInterval, axis = 0)

    return trainingdf, testdf

def accuracy(df, correctlabels):
    dfnew = df.copy()
    maxIndex = dfnew.idxmax(axis=1)
    #print(maxIndex)
    #print(len(dfnew.columns))
    
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for i, elem in enumerate(maxIndex):
        if maxIndex[i] == correctlabels[i]:
            tp += 1
        else:
            fp += 1
        
    accuracy = (tp+tn)/(tp+fp+tn+fn)#(len(dfnew.columns) * len(dfnew.index))
    #print("counter: {}".format(counter))
    #print("tp: {}, tn: {}, fp: {}, fn: {}".format(tp, tn, fp, fn))

    #print(maxIndex)
    return accuracy

def folds(df, nofolds):
    dfnew = df.copy()
    output = []
    dflist = np.random.permutation(df.index)
    
    splits = np.array_split(dflist, nofolds, axis=0)
    output = []
    for subArray in splits:
        # print(subArray)
        tempdf = pd.DataFrame(subArray)
        output.append(tempdf)
    return output

def brier_score(df, correctlabels):
    dfnew = df.copy()

    #print(correctlabels)
    #display(df)
    
    counter = 0
    score = 0
    for i, row in enumerate(dfnew.iterrows()):
     
        correctIndex = np.where(df.columns==correctlabels[i])[0][0]
        #print("correctIndex: {}".format(correctIndex))

        for j, col in enumerate(row[1]):
            counter += 1
            #print("col: {}".format(col))


            if j == correctIndex:
                score += (col-1)**2
            else:
                score += (col)**2

    
    # score /= counter

    return score/df.index.size

def auc(df, correctlabels):


    dfnew = df.copy()
  

    guesses = dfnew.idxmax(axis=1) #används ej?
   
    weights = []
    for i, col in enumerate(dfnew.columns):
        weights.append(correctlabels.value_counts() / len(correctlabels))

    #hint3 list of dictionaries?
    dictlist = []

    for col in dfnew.columns:
        dictlist.append({})

    for i, row in enumerate(dfnew.iterrows()):
        for j, col in enumerate(row[1]):            
            key = str(col)
            pos, neg = dictlist[j].get(key, (0,0))

            if row[1].index[j] == correctlabels[i]:
                dictlist[j][key] = (pos+1, neg)
            else:
                dictlist[j][key] = (pos, neg+1)

    for i in range(len(dictlist)):
        dictlist[i] = {key: val for key, val in sorted(dictlist[i].items(), key = lambda x: x[0], reverse=True)}
        #print("dictionary {}: {}".format(i, dictlist[i]))

    #hint 4.2, iteration

    aucList = []
    printListAucs = []
    for i, colDict in enumerate(dictlist):

        colArea = 0
        tottp = 0
        totfp = 0
        for tup in colDict.values():
            tottp += tup[0]
            totfp += tup[1]

        #print("tp {} fp {}".format(tottp, totfp))

        savedHeight = 0
        for tp, fp in colDict.values():
            
            if tp != 0 and fp != 0 and savedHeight == 0:
                #triangle utan extra höjd
                colArea += (fp/totfp * tp/tottp)/2 #basen ggr höjden delat på 2

                savedHeight += tp/tottp
            
            elif tp != 0 and fp != 0 and savedHeight != 0:
                # om vi har tidigare höjd, dela upp till rekt + trek 

                #triangle
                colArea += (fp/totfp * tp/tottp)/2 #basen ggr höjden delat på 2

                # rekt B)
                colArea += fp/totfp * savedHeight

                #öka höjd med trek
                savedHeight += tp/tottp
            
            elif fp == 0:
                #bara tp
                # ha höjd i åtanke till nästa, inte förra
                savedHeight += tp/tottp

            elif tp == 0:
                #bara fp
                # räkna med föregående höjd, ta inte med till nästa
                colArea += fp/totfp * savedHeight

        
        printListAucs.append(colArea)
        aucList.append(colArea*weights[i])

    #print("col aucs: {}\nweights: {}".format(printListAucs, weights))

    #print("aucList AAAAAAAAAAAAAAAAH")
    #print(aucList)
    #print("should summywum")
    #print(np.sum(aucList))
    
    return np.sum(aucList)

## 1. Define the class kNN

In [4]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies
import time 



class kNN:

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.training_labels = None
        self.labels = None
        self.training_data = None

    def fit(self, df, normalizationtype='minmax'):
        #display(df)
        start = time.time()
        
        dfnew, self.column_filter = create_column_filter(df)
        dfnew, self.imputation = create_imputation(dfnew)
        dfnew, self.normalization = create_normalization(dfnew, normalizationtype)
        dfnew, self.one_hot = create_one_hot(dfnew)
        dfnew['CLASS'] = dfnew['CLASS'].astype('category')
        self.training_labels = dfnew['CLASS']
        self.labels = pd.unique(sorted(df['CLASS'].astype("category")))
        self.training_data = dfnew.drop(columns = ['CLASS', 'ID']).to_numpy()
        print("fit time: {}".format(time.time()-start))

        
    def predict(self, df, k=5):

        dfnew = df.drop(columns = ['CLASS', 'ID'])
        dfnew = apply_column_filter(dfnew, self.column_filter)
        dfnew = apply_imputation(dfnew, self.imputation)
        dfnew = apply_normalization(dfnew, self.normalization)
        dfnew = apply_one_hot(dfnew, self.one_hot) #will not be used in our instance because our test is not objects or categorilcals
        # predictions = pd.DataFrame()
        

    
        # def relative_frequency(sort_distance, predictdf, i):
        #     frequencies = np.zeros(len(self.training_labels))
        #     for dist, c in sort_distance:
        #         predictdf.iloc[i, c-2] += 1

        #     frequencies = frequencies/k
            # return frequencies
        
        # def get_nearest_neighbour(self, x_test, k): 
        #     distanceArr = []

        #     for j, row in enumerate(self.training_data):
        #         distance = 0.0
        #         for i, element in enumerate(row):
        #             distance += np.linalg.norm(element-x_test[i])
        #             distanceArr.append((distance, self.training_labels[j]))
                    
        #     output = sorted(distanceArr)
        #     return output[:k]
        
        # skapa en df med olumner enligt de class lables vi har i original df (moddad self.labels)
        predictionsdf = pd.DataFrame(columns=[self.labels], index=dfnew.index)
        predictionsdf = predictionsdf.fillna(0.0)
        
        def get_nearest_neighbor_predictions(self, x_test, k):
            # key=index value=distance series
            distances = {}
            for i, row in enumerate(self.training_data):
                distance = np.sqrt(np.sum( (row - x_test)**2 ))
                distances[i] = distance
        
            distances = sorted(distances.items(), key=lambda x:x[1])
            k_nearest_labels = np.array([self.training_labels[distances[i][0]] for i in range(k)])
            return k_nearest_labels

        # for each row in original df
        for i, row in enumerate(dfnew.iterrows()):
            start = time.time()
            # calculate nearest neighbors
            nearest_neighbors = get_nearest_neighbor_predictions(self, row[1], k)
           
        # for each nearest neighbor, increment their class probability by 1 at new predictiondf row
        # for (distance, label) in nearest_neighbors:
            
            for label in nearest_neighbors:
        
                # select rad = i, column = label
                
                predictionsdf.loc[i, label] += 1
               
            
            # also, when all neighbors are added, divide the row by number of neighbors
            
            predictionsdf.loc[i] = predictionsdf.loc[i].div(k)
        

   
        return predictionsdf
    
    

            
            
            
            


        




In [26]:
#TEST, DELETE
#TEST, DELETE
#TEST, DELETE
#TEST, DELETE
#TEST, DELETE
#TEST, DELETE
#TEST, DELETE
#TEST, DELETE
#TEST, DELETE

import time 
class kNN:

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.training_labels = None
        self.labels = None
        self.training_data = None

    def fit(self, df, normalizationtype='minmax'):
        #display(df)
        start = time.time()
        
        dfnew, self.column_filter = create_column_filter(df)
        dfnew, self.imputation = create_imputation(dfnew)
        dfnew, self.normalization = create_normalization(dfnew, normalizationtype)
        dfnew, self.one_hot = create_one_hot(dfnew)
        dfnew['CLASS'] = dfnew['CLASS'].astype('category')
        self.training_labels = dfnew['CLASS']
        self.labels = pd.unique(sorted(df['CLASS'].astype("category")))
        self.training_data = dfnew.drop(columns = ['CLASS', 'ID']).to_numpy()
        print("fit time: {}".format(time.time()-start))

        
    def predict(self, df, k=5):

        dfnew = df.drop(columns = ['CLASS', 'ID'])
        dfnew = apply_column_filter(dfnew, self.column_filter)
        dfnew = apply_imputation(dfnew, self.imputation)
        dfnew = apply_normalization(dfnew, self.normalization)
        dfnew = apply_one_hot(dfnew, self.one_hot) #will not be used in our instance because our test is not objects or categorilcals
        # predictions = pd.DataFrame()
        
        # skapa en df med olumner enligt de class lables vi har i original df (moddad self.labels)
        predictionsdf = pd.DataFrame(columns=[self.labels], index=dfnew.index)
        predictionsdf = predictionsdf.fillna(0.0)
        
        def get_nearest_neighbor_predictions(self, x_test, k):
            # key=index value=distance series
            distances = []
            for i, row in enumerate(self.training_data):
                distance = np.sqrt(np.sum( (row - x_test)**2 ))
                distances.append(distance)
        
#             distances = sorted(distances.items(), key=lambda x:x[1])
            distances = np.argsort(distances)
            nn_labels = np.array([self.training_labels[distances[i]] for i in range(k)])
#             k_nearest_labels = np.array([self.training_labels[distances[i][0]] for i in range(k)])
            return nn_labels

        # for each row in original df
        for i, row in enumerate(dfnew.iterrows()):
            
            # calculate nearest neighbors
            nearest_neighbors = get_nearest_neighbor_predictions(self, row[1], k)
           
        # for each nearest neighbor, increment their class probability by 1 at new predictiondf row
        # for (distance, label) in nearest_neighbors:
            
            for label in nearest_neighbors:
        
                # select rad = i, column = label
                
                predictionsdf.loc[i, label] += 1
               
            
            # also, when all neighbors are added, divide the row by number of neighbors
            
            predictionsdf.loc[i] = predictionsdf.loc[i].div(k)
        

   
        return predictionsdf
    
    

            
            
            
            


        




In [27]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),2))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score"])

print()
display("results",results)

fit time: 0.007006406784057617
Training time: 0.01 s.
Testing time (k=1): 1.79 s.
Testing time (k=3): 1.88 s.
Testing time (k=5): 1.99 s.
Testing time (k=7): 2.10 s.
Testing time (k=9): 2.19 s.



'results'

,Accuracy,Brier score
1,0.747664,0.504673
3,0.663551,0.488058
5,0.579439,0.471028
7,0.598131,0.471867
9,0.616822,0.482981


In [7]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 6.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [8]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of 
#                                   (non-missing, categorical) value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some 
#                                   (non-missing, categorical) value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from 
#         a column name (a specific feature) to another mapping, which given a class label and a value for 
#         the feature, returns the number of training instances which have included this combination, 
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you 
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.

class NaiveBayes:
    def __init__ (self):
        self.column_filter = None;
        self.binning = None;
        self.class_priors = None;
        self.labels = None;
        self.feature_class_value_counts = None;
        self.feature_class_counts=None;
    
    def fit (self, df, nobins=10, bintype='equal-width'):
        
        dfnew, self.column_filter = create_column_filter(df)
        dfnew, self.binning = create_bins(dfnew, nobins, bintype)
        self.class_priors = df['CLASS'].value_counts(normalize=True).to_dict()
        self.labels = pd.unique(sorted(df['CLASS'].astype("category")))
        self.feature_class_value_counts = {}
        self.feature_class_counts = {}

        for col in dfnew.columns:
            if col ==  'ID' or col == 'CLASS':
                continue
            self.feature_class_counts[col] = dfnew['CLASS'].value_counts().to_dict()
            groupedcolumns = dfnew.groupby(["CLASS", col]).size()
            self.feature_class_value_counts[col]=groupedcolumns.to_dict()
        #laplace correction for feature_class_counts
        # for feature in self.feature_class_counts:
        #     # print(feature)
        #     for cl in self.feature_class_counts[feature]:
        #         self.feature_class_counts[feature][cl] += 1
        
        #laplace correction for feature_class_value_counts
        # for feature in self.feature_class_value_counts:
        #     # print(feature)
        #     for cl in self.feature_class_value_counts[feature]:
        #         # self.feature_class_value_counts[feature][cl] += 1
        #         self.feature_class_value_counts[feature][cl] += 1

        # # print(self.feature_class_value_counts)
        # print(self.feature_class_counts)
        # print("feature class value counts", self.feature_class_counts)

    
    def predict (self, df):
        dfnew = df.copy()
        dfnew = apply_column_filter(dfnew, self.column_filter)
        dfnew = apply_bins(dfnew, self.binning)
        dfnew = dfnew.drop(columns = ["ID", "CLASS"])
        # len(self.binning[self.labels[0]])-1)  
        # print(self.feature_class_value_counts)
        # print(self.feature_class_counts)
        
        classProbs = []
        rowlength = df.shape[0]
        columnlength = df.shape[1]
        labellength = len(self.labels)
        matrix = np.zeros([labellength,rowlength,columnlength])
        for columnIndex, col in enumerate(dfnew.columns):
            for labelIndex, cl in enumerate(self.labels):
                rfreqlist = []
                for rowIndex in range(rowlength):
                # for feature in dfnew[col]
                    value = dfnew.iloc[rowIndex, columnIndex]
                    if((cl, value) in self.feature_class_value_counts[col].keys()):

                        label_count = self.feature_class_counts[col][cl]
                        feature_value_count = self.feature_class_value_counts[col][(cl, value)]
                        rfreq = feature_value_count/label_count

                    # print("feature_value_count: {}".format(feature_value_count))
                    # print("label count: {}".format(label_count))
                    # rfreqlist.append(rfreq) 
                    else:
                        rfreq = 0                 

                    print("rfreq", rfreq)
                    # classProbs.append(np.prod(rfreqlist) * self.class_priors[cl])
                matrix[labelIndex, rowIndex, columnIndex] = rfreq
        prodmatrix = matrix.prod(axis=2)
        print("prodmatrix", prodmatrix)




        
        # print("Col: {}, Probability for class {} is: {}".format(col, cl, classProbs))
        denom = np.sum(classProbs)
        normed = None
        if denom != 0:
            normed = classProbs / denom
        else:
            normed = self.class_priors 

        print(len(self.class_priors))

                # TODO: kontrollera ord för ord ish hint 2. "observed feature"
                # vi tar ju bara alla möjliga, ska vara chill
                # tänk edvins geniala "unika kombination av klass och bin" blir inte 7*106
                # kolla också var hints går över till att prata probabilities.
                # är det verkligen det vi har? summerar inte till 1


        

        
        


In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)
# TEST RUTA
# TEST RUTA

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.04 s.
rfreq 0.6129032258064516
rfreq 0.0
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.3870967741935484
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.3870967741935484
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.6129032258064516
rfreq 0.0
rfreq 0.61290

rfreq 0.375
rfreq 0.625
rfreq 0.375
rfreq 0.625
rfreq 0.375
rfreq 0.625
rfreq 0.625
rfreq 0.375
rfreq 0.625
rfreq 0.625
rfreq 0.375
rfreq 0.625
rfreq 0.625
rfreq 0.625
rfreq 0.0
rfreq 0.625
rfreq 0.375
rfreq 0.625
rfreq 0.375
rfreq 0.625
rfreq 0.625
rfreq 0.625
rfreq 0.375
rfreq 0.625
rfreq 0.0
rfreq 0.375
rfreq 0.625
rfreq 0.625
rfreq 0.625
rfreq 0.625
rfreq 0.625
rfreq 0.625
rfreq 0.75
rfreq 0.0
rfreq 0.0
rfreq 0.0
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.75
rfreq 0.0
rfreq 0.0
rfreq 0.75
rfreq 0.25
rfreq 0.75
rfreq 0.0
rfreq 0.0
rfreq 0.0
rfreq 0.0
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.25
rfreq 0.75
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.25
rfreq 0.75
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.75
rfreq 0.0
rfreq 0.75
rfreq 0.0
rfreq 0.75


AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
# alex testar
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of 
#                                   (non-missing, categorical) value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some 
#                                   (non-missing, categorical) value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from 
#         a column name (a specific feature) to another mapping, which given a class label and a value for 
#         the feature, returns the number of training instances which have included this combination, 
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you 
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.

class NaiveBayes:
    def __init__ (self):
        self.column_filter = None;
        self.binning = None;
        self.class_priors = None;
        self.labels = None;
        self.feature_class_value_counts = None;
        self.feature_class_counts=None;
    
    def fit (self, df, nobins=10, bintype='equal-width'):
        
        dfnew, self.column_filter = create_column_filter(df)
        dfnew, self.binning = create_bins(dfnew, nobins, bintype)
        self.class_priors = df['CLASS'].value_counts(normalize=True).to_dict()
        self.labels = pd.unique(sorted(df['CLASS'].astype("category")))
        self.feature_class_value_counts = {}
        self.feature_class_counts = {}

        for col in dfnew.columns:
            if col ==  'ID' or col == 'CLASS':
                continue
            self.feature_class_counts[col] = dfnew['CLASS'].value_counts().to_dict()
            groupedcolumns = dfnew.groupby(["CLASS", col]).size()
            self.feature_class_value_counts[col]=groupedcolumns.to_dict()

        #laplace correction for feature_class_counts
        # for feature in self.feature_class_counts:
        #     # print(feature)
        #     for cl in self.feature_class_counts[feature]:
        #         self.feature_class_counts[feature][cl] += 1
        
        #laplace correction for feature_class_value_counts
        # for feature in self.feature_class_value_counts:
        #     # print(feature)
        #     for cl in self.feature_class_value_counts[feature]:
        #         # self.feature_class_value_counts[feature][cl] += 1
        #         self.feature_class_value_counts[feature][cl] += 1

        # # print(self.feature_class_value_counts)
        # print(self.feature_class_counts)
        #print("feature class value counts", self.feature_class_value_counts["Ba"])

    
    def predict (self, df):
        dfnew = df.copy()
        dfnew = apply_column_filter(dfnew, self.column_filter)
        dfnew = apply_bins(dfnew, self.binning)
        dfnew = dfnew.drop(columns = ["ID", "CLASS"])
        # len(self.binning[self.labels[0]])-1)  
        # print(self.feature_class_value_counts)
        # print(self.feature_class_counts)
        #print(pd.unique(dfnew["Ba"]))
        
        # create skeleton for new DF
        predictionsdf = pd.DataFrame(columns=self.labels, index=dfnew.index)
        predictionsdf = predictionsdf.fillna(0.0)
        #display(predictionsdf)
        #print(predictionsdf.loc[0,1])

        for i, row in enumerate(dfnew.iterrows()):
            classProbs = []
            for cl in self.labels:
                rfreqlist = []
                #for feature in range(len(self.binning[col])-1):
                for j, feature in enumerate(row[1]):
                    col = dfnew.columns[j]
                    #print(feature)
                    label_count = self.feature_class_counts[col][cl]
                    #print("col: {}, feature: {}, cl: {}".format(col, feature, cl))
                    try:
                        feature_value_count = self.feature_class_value_counts[col][(cl, feature)]

                    except:
                        continue
                
                    rfreq = feature_value_count/label_count

                    # print("feature_value_count: {}".format(feature_value_count))
                    # print("label count: {}".format(label_count))
                    rfreqlist.append(rfreq)                  

                    # print("rfreq", rfreq)
                classProb = (np.prod(rfreqlist) * self.class_priors[cl])
                #print("i, cl: {}, {}".format(i, cl))
                #print(classProb)
                predictionsdf.loc[i, cl] = classProb
                classProbs.append(classProb)
            #gör div här?
            #print("sum: {}", np.sum(classProbs))
            #print("row bef {}: {}".format(i, predictionsdf.loc[i]))
            if np.sum(classProbs) != 0:
                predictionsdf.loc[i] /= np.sum(classProbs)
            else:
                #ööööh set to class priors
                print("not implemented class priors yet")
                return 1
            #print("row aft {}: {}".format(i, predictionsdf.loc[i]))
            #print("sum for row: {}", np.sum(predictionsdf.loc[i]))
        #display(predictionsdf) 

        return predictionsdf


In [ ]:

#alex körruta
# Test your code (leave this part unchanged, except for if auc is undefined)
# TEST RUTA
# TEST RUTA

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

In [ ]:
# ================================================ # COLUMN APPROACH

        # for each column in original df, col = NA/SI/FE...
        for col in dfnew.columns:

            # for each label in self.labels (label=class labels, 1, 2, 3, 5, 6, 7)
            for label in self.labels:
                relative_frequencies = []
                nn_class_probabilities = []

                # calculate relative frequency of (unique) observed feature value given the class,
                # ACCORDING TO PERCENTAGES/FREQUENCIES IN TRAINING MODEL!
                for value in pd.unique(dfnew[col]):
                    label_frequency = self.feature_class_counts[col][label]
                    label_value_frequency = self.feature_class_value_counts[col][(label, value)]
                    print("in {}, label {} occurs {} times, and value {} represents {} of these occurences".format(col, label, label_frequency, value, label_value_frequency))
                    relative_frequency = label_value_frequency/label_frequency
                    print("      i.e. value {} represents {} of the total label {} occurences".format(value, relative_frequency, label))
                    relative_frequencies.append(relative_frequency)

                # ????????????????????????????????????????????????
                nn_class_probabilities.append(np.prod(relative_frequencies) * self.class_priors[label])
                print("relative frequencies: ", relative_frequencies)
                print("classProbability: ", np.prod(relative_frequencies) * self.class_priors[label])
                print(" ")

In [ ]:
# EXTRA RUTA, DELETE ============================================================
# EXTRA RUTA, DELETE ============================================================
# EXTRA RUTA, DELETE ============================================================
# EXTRA RUTA, DELETE ============================================================
# EXTRA RUTA, DELETE ============================================================
# EXTRA RUTA, DELETE ============================================================
# EXTRA RUTA, DELETE ============================================================
# EXTRA RUTA, DELETE ============================================================

# print("label: {}, label frequency for {}: {}, value: {}, label value frequency: {}".format(label, col, label_frequency, value, label_value_frequency))

class NaiveBayes:
    def __init__ (self):
        self.column_filter = None;
        self.binning = None;
        self.class_priors = None;
        self.labels = None;
        self.feature_class_value_counts = None;
        self.feature_class_counts=None;
    
    def fit (self, df, nobins=10, bintype='equal-width'):  
        dfnew, self.column_filter = create_column_filter(df)
        dfnew, self.binning = create_bins(dfnew, nobins, bintype)
        self.class_priors = df['CLASS'].value_counts(normalize=True).to_dict()
        self.labels = pd.unique(sorted(df['CLASS'].astype("category")))
        self.feature_class_value_counts = {}
        self.feature_class_counts = {}

        for col in dfnew.columns:
            if col == 'CLASS' or col == 'ID':
                continue
            self.feature_class_counts[col] = dfnew['CLASS'].value_counts().to_dict()
            groupedcolumns = dfnew.groupby(['CLASS', col]).size()
            
            self.feature_class_value_counts[col]=groupedcolumns.to_dict()
        
        print("self labels: ", self.labels)
        print("class value count:", self.feature_class_value_counts)
        print("class count:", self.feature_class_counts)
        print("class priors:", self.class_priors)


    def predict(self, df):
        # apply filter & discretization
        dfnew = df.copy()
        dfnew = dfnew.drop(columns = ['ID', 'CLASS'])
        dfnew = apply_column_filter(dfnew, self.column_filter)
        dfnew = apply_bins(dfnew, self.binning)
        display(dfnew)

        # create skeleton for new DF
        predictionsdf = pd.DataFrame(columns=self.labels, index=dfnew.index)
        predictionsdf = predictionsdf.fillna(0.0)
        #display(predictionsdf)

        # for each row in original df
        for i, row in enumerate(dfnew.iterrows()):

            # for each label
            for label in self.labels:
                relative_frequencies = []

                # for each value in said row
                for j, value in enumerate(row[1]):
                    #print(dfnew.columns[j])
                    col = dfnew.columns[j]
                    #print("row:{}, col:{}, label:{}, value:{}".format(i, col, label, value))
                    label_frequency = self.feature_class_counts[col][label]
                    label_value_frequency = self.feature_class_value_counts[col][(label, value)]
                    relative_frequencies.append(label_value_frequency/label_frequency)

                #predictionsdf.loc[i, label] = np.prod(relative_frequencies)*self.class_priors[label]

        #display(predictionsdf)
        return predictionsdf


In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

In [ ]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

### Comment on assumptions, things that do not work properly, etc.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=22883318-2b19-4ed9-97c4-61f4bb16ce59' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>